## QuantLib - Bonds and Spot/Discount Curves

In [ ]:
#!pip install Quantlib-Python

In [1]:
# Calculate the present value of a bond's cashflows based on spot rates
# using lists/math as well as using QuantLib

In [2]:
# represent the bond's flows as a list of (flow, time-in-years)
flows = [(.025, .5), (.025, 1), (.025, 1.5), (1.025, 2.0)] # 2yr 5% semi-ann
flows

[(0.025, 0.5), (0.025, 1), (0.025, 1.5), (1.025, 2.0)]

In [3]:
# The spot rates are a list (parallel with the flows) of annual continuously compounded rates
spotRates1 = [.02, .03, .04, .045]
spotRates1

[0.02, 0.03, 0.04, 0.045]

In [4]:
# flows are discounted/summed to calculate a Net Present Value, based on spot rates
import math
npv = 0
i = 0
print('t  ', 'flow ', 'spot', 'discount', 'pv')
for flow in flows:
    t = flow[1]
    #discountFactor = 1 / math.exp(spotRates[i] * t) # this is for continuously compounded rates
    discountFactor = 1 / pow(1+spotRates1[i],t) # this is for annual rates
    pv = flow[0] * discountFactor
    print("%.1f" % t, flow[0], "%.2f" % (spotRates1[i]*100), "%.6f" % discountFactor, "%.6f" % pv)
    npv += pv
    i += 1
print("spot-based price=", "%.3f" % (npv*100))

t   flow  spot discount pv
0.5 0.025 2.00 0.990148 0.024754
1.0 0.025 3.00 0.970874 0.024272
1.5 0.025 4.00 0.942866 0.023572
2.0 1.025 4.50 0.915730 0.938623
spot-based price= 101.122


In [5]:
# using QuantLib
# see http://gouthamanbalaraman.com/blog/quantlib-bond-modeling.html
# first define the term structure based on zero rates (at cashflow points)
import QuantLib as ql
todaysDate = ql.Date(31, 12, 2018)
ql.Settings.instance().evaluationDate = todaysDate
spotDates = [ql.Date(31, 12, 2018),
             ql.Date(30, 6, 2019), ql.Date(31, 12, 2019),
             ql.Date(30, 6, 2020), ql.Date(31, 12, 2020)]
spotRates2 = [0, 0.02, 0.03, .04, .045]
dayCount = ql.Thirty360(ql.Thirty360.USA)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual
spotCurve = ql.ZeroCurve(spotDates, spotRates2, dayCount, calendar, interpolation,
                             compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)
type(spotCurveHandle)
for method in dir(spotCurveHandle):
    if method[0] != "_": print(method, end=" ")

allowsExtrapolation asObservable calendar dayCounter disableExtrapolation discount enableExtrapolation forwardRate maxDate maxTime referenceDate this thisown zeroRate 

In [6]:
# define the bond's payment schedule
issueDate = ql.Date(31, 12, 2018)
maturityDate = ql.Date(31, 12, 2020)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
bussinessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, bussinessConvention,
                            bussinessConvention , dateGeneration, monthEnd)
print(list(schedule))

[Date(31,12,2018), Date(30,6,2019), Date(31,12,2019), Date(30,6,2020), Date(31,12,2020)]


In [7]:
# define the bond
dayCount = ql.Thirty360(ql.Thirty360.USA)
couponRate = .05
coupons = [couponRate]
settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)
#[f for f in dir(fixedRateBond) if f[0] != '_']

In [8]:
# set up a bond engine, using the previously constructed spot curve
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)

In [9]:
# calculate the theoretical (full) price of the bond, based on the spot curve
fixedRateBond.setPricingEngine(bondEngine)
print("%.3f" % fixedRateBond.NPV())

101.122
